In [626]:
import numpy as np

boardSize=8
PopSize = 200
generations = 30
mutationPerc = 1
initPop =  np.zeros(shape=(PopSize,boardSize)) 
population = np.zeros(shape=(initPop.shape[0], 8, 8))

for y in range(0, PopSize):
    Pop = [1,2,3,4,5,6,7,0]
    for x in range(0,boardSize-1): #Random Shuffle of 8 queens
        rn =np.random.randint(boardSize - x)
        r= x-(rn)
        a = Pop[r]
        Pop[r] =Pop[x]
        Pop[x] = a
    initPop[y, :] = Pop


In [627]:
def dim2to3(cross):
    pop  = np.zeros(shape=(cross.shape[0], 8, 8))
    x = 0
    for x in  range(len(cross)):
        for i in range(len(cross[0])):
            pop[x,  np.uint8(cross[x][i]),i ] = 1 # 2D to 3D
    return (pop )


In [628]:
population = dim2to3(initPop)
print ("First Indices \n", initPop[0])
print ("First Board \n",population[0])


First Indices 
 [ 1.  2.  5.  6.  4.  3.  7.  0.]
First Board 
 [[ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]]


In [629]:
def queensColumn(  pop):
    tcattcks = np.zeros(pop.shape[0])  # Number of attacks for all solutions (column only).
    for xpos in range(pop.shape[0]):
        temp = pop[xpos, :]
        for ypos in range(boardSize):
            col_sum = np.sum(temp[:, ypos])
            if (col_sum == 0 or col_sum == 1):
                col_sum = 0
            else:
                col_sum = col_sum - 1   
        
            tcattcks[xpos] = tcattcks[i] + col_sum

    return np.power(tcattcks,3 )

In [630]:
t = queensColumn(population)
print ("column queens",t[0])

column queens 0.0


In [631]:
def diagonal_attacks(  mat):
     if (mat.shape[0] < 2 or mat.shape[1] < 2):
         return 0
     nda = mat.diagonal().sum() - 1
     return nda

In [632]:
def queensdiagonal( pop):
    total_num_attacks = np.zeros(pop.shape[0])  # Number of attacks for all solutions (diagonal only).

    for solution_idx in range(pop.shape[0]):
         ga_solution = pop[solution_idx, :]
         temp = np.zeros(shape=(10, 10))
         temp[1:9, 1:9] = ga_solution
         row_indices, col_indices = np.where(ga_solution == 1)
         row_indices = row_indices + 1
         col_indices = col_indices + 1
         total = 0  # total number of attacking pairs diagonally for each solution.

         for element_idx in range(8):
             x = row_indices[element_idx]
             y = col_indices[element_idx]

             mat_bottom_right = temp[x:, y:]
             total = total +  diagonal_attacks(mat_bottom_right)

             mat_bottom_left = temp[x:, y:0:-1]
             total = total +  diagonal_attacks(mat_bottom_left)

             mat_top_right = temp[x:0:-1, y:]
             total = total +  diagonal_attacks(mat_top_right)

             mat_top_left = temp[x:0:-1, y:0:-1]
             total = total +  diagonal_attacks(mat_top_left)

         # Dividing the total by 2 because it counts the solution as attacking itself diagonally.
         total_num_attacks[solution_idx] = total_num_attacks[solution_idx] + total / 2

    return np.power(total_num_attacks,3)

In [633]:
def fit( p  ):
    tattacks =  queensdiagonal( pop= p) +  queensColumn( p)
    pfit = np.copy(
        tattacks)  

    for solution_idx in range(p.shape[0]):
        if pfit[solution_idx] == 0:
            pfit[solution_idx] = float("inf")
        else:
            pfit[solution_idx] = 1.0 / pfit[solution_idx]
    return pfit , tattacks  

In [634]:
population_fitness,total_num_attacks= fit( population)


In [635]:
def rank_selection( fitness, num_parents):
    fitness_sorted = sorted(range(len(fitness)), key=lambda k: fitness[k])
    fitness_sorted.reverse()
    parents = np.empty(shape=(num_parents, initPop.shape[1]))
    for parent_num in range(num_parents):
       parents[parent_num, :] = initPop[fitness_sorted[ parent_num], :]
    return parents  

In [636]:
parents = rank_selection (population_fitness,2)
print (parents)

[[ 0.  5.  2.  6.  3.  7.  4.  1.]
 [ 4.  0.  3.  6.  2.  5.  7.  1.]]


In [637]:
def single_point_crossover(  parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = np.random.randint(low=0, high=parents.shape[1], size=1)[0]

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k % parents.shape[0]

        # Index of the second parent to mate.
        parent2_idx = (k+1) % parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
        #print ("offspring \n",offspring)
    return offspring
    

In [638]:
cross =   single_point_crossover(parents,parents.shape)

In [639]:
def swap_mutation(offspring):
    for idx in range(offspring.shape[0]):
        mutation_gene1 = np.random.randint(low=0, high=offspring.shape[1]/2, size=1)[0]
        mutation_gene2 = mutation_gene1 + int(offspring.shape[1]/2)

        temp = offspring[idx, mutation_gene1]
        offspring[idx, mutation_gene1] = offspring[idx, mutation_gene2]
        offspring[idx, mutation_gene2] = temp
    return offspring


In [640]:
cross =   swap_mutation(cross )
print(cross)

[[ 2.  0.  3.  6.  0.  5.  7.  1.]
 [ 4.  5.  4.  6.  3.  7.  2.  1.]]


In [641]:
    cross2dto3d = dim2to3(cross)
    population = np.append(population, cross2dto3d, axis = 0)
    initPop = np.append(initPop, cross ,axis = 0)
   

In [642]:
generations = 500
mutationPerc=50
population_fitness,attacks =  fit(population)
for i in range(  generations):
    print ("Generation " , i)
    best_match_idx = np.where(population_fitness == np.max(population_fitness))[0][0]
    best_solution =  initPop[best_match_idx, :]
    best_solution_fitness = population_fitness[best_match_idx]
    #print ("Pop Fitness Score:\n{} attacks \n{} \nPopulation:\n{}\n".format(population_fitness,attacks, initPop))
    print ("best_match_idx {} Max Fitness Score: {}   attacks {} \nSolution:\n{}\n ".format(best_match_idx,best_solution_fitness,attacks[best_match_idx], best_solution))
    if best_solution_fitness == float("inf"):
        break #return best_solution, best_solution_fitness, best_match_idx
    parents =  rank_selection( population_fitness,np.uint8(len(population_fitness)/4) ) 
    #parents =  rank_selection( population_fitness,5 )
    
    cross =   single_point_crossover(parents, parents.shape)
    if np.random.randint(1,100) <=  mutationPerc:
        cross =  swap_mutation( cross)
        print ("mutate applied")

    initPop = np.append(initPop, cross ,axis = 0)
    cross2dto3d = dim2to3(cross)
    population = np.append(population, cross2dto3d, axis = 0)
    pf,att =  fit(cross2dto3d)
    population_fitness =np.append(population_fitness,pf, axis = 0)
    attacks =np.append(attacks,att, axis = 0)

 
    

Generation  0
best_match_idx 41 Max Fitness Score: 1.0   attacks 1.0 
Solution:
[ 4.  7.  0.  2.  5.  1.  6.  3.]
 
mutate applied
Generation  1
best_match_idx 41 Max Fitness Score: 1.0   attacks 1.0 
Solution:
[ 4.  7.  0.  2.  5.  1.  6.  3.]
 
mutate applied
Generation  2
best_match_idx 295 Max Fitness Score: inf   attacks 0.0 
Solution:
[ 2.  5.  7.  0.  4.  6.  1.  3.]
 


In [643]:
print(initPop)

[[ 1.  2.  5. ...,  3.  7.  0.]
 [ 0.  6.  4. ...,  7.  3.  1.]
 [ 1.  5.  6. ...,  2.  7.  0.]
 ..., 
 [ 3.  2.  5. ...,  7.  4.  0.]
 [ 0.  4.  1. ...,  2.  7.  6.]
 [ 2.  5.  3. ...,  0.  7.  1.]]


In [644]:

print(initPop[best_match_idx])

[ 2.  5.  7.  0.  4.  6.  1.  3.]


In [645]:
print(attacks[best_match_idx])

0.0


In [646]:
a = queensColumn(population)
print(a[best_match_idx])

0.0


In [647]:
a = queensdiagonal(population)
print(a[best_match_idx])

0.0


In [648]:
print(population[best_match_idx])

[[ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]]


In [650]:
print(initPop[best_match_idx])

[ 2.  5.  7.  0.  4.  6.  1.  3.]


In [683]:
import requests 
url='https://lf8q0kx152.execute-api.us-east-2.amazonaws.com/default/computeFitnessScore' 
x=requests.post(url,json={"qconfig":" ","userID":169130,"githubLink":" "}) 
print(x.text)


SSLError: ("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",)

In [681]:

print('{:d}'.format(initPop[best_match_idx][0]))

ValueError: Unknown format code 'd' for object of type 'float'